In [14]:
import pandas as pd
import numpy as np
from sklearn import datasets
# import plotly as py
# import plotly.plotly as plt
# import plotly.graph_objs as go
# from plotly import __version__
from sklearn import preprocessing
#from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
#py.tools.set_credentials_file(username='j.carpenter_12', api_key='t07tLoWdgxKH9sS8TH1T')


In [15]:
boston = datasets.load_boston()
names = boston.feature_names
target = boston.target

bos = pd.DataFrame(boston.data)
bos.columns = names
bos.head()


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [16]:
trace = go.Scatter(x=bos.RM,
                   y=target,
                   mode ='markers')

NameError: name 'go' is not defined

In [17]:
plt.iplot([trace])

NameError: name 'plt' is not defined

In [18]:
####Normalize data
val = bos.values
min_max_scaler = preprocessing.MinMaxScaler()
val_scaled = min_max_scaler.fit_transform(val)
bos_norm = pd.DataFrame(val_scaled,columns=bos.columns)
bos_norm.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.000000,0.18,0.067815,0.0,0.314815,0.577505,0.641607,0.269203,0.000000,0.208015,0.287234,1.000000,0.089680
1,0.000236,0.00,0.242302,0.0,0.172840,0.547998,0.782698,0.348962,0.043478,0.104962,0.553191,1.000000,0.204470
2,0.000236,0.00,0.242302,0.0,0.172840,0.694386,0.599382,0.348962,0.043478,0.104962,0.553191,0.989737,0.063466
3,0.000293,0.00,0.063050,0.0,0.150206,0.658555,0.441813,0.448545,0.086957,0.066794,0.648936,0.994276,0.033389
4,0.000705,0.00,0.063050,0.0,0.150206,0.687105,0.528321,0.448545,0.086957,0.066794,0.648936,1.000000,0.099338


In [19]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split
X= bos_norm
y= target
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.15,random_state=42)

In [20]:
lr = linear_model.LinearRegression()

In [21]:
lr.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [22]:
lr.score(X_train,y_train)


0.7320210474402874

In [23]:
###plot lr prediction intercept

intercept = lr.intercept_
coef = lr.coef_[0]
x = X_test['RM']
min_x = min(x)
max_x = max(x)

fit = [intercept, coef*max_x + intercept]

df = pd.DataFrame({'Predicted':lr.predict(X_test),'Actual':y_test})


z = np.polyfit(x=df.loc[:,'Predicted'],y=df.loc[:,'Actual'],deg=1)
p = np.poly1d(z)
df['trendline'] = p(df.loc[:,'Predicted'])



trace1 = go.Scatter(x=df['Predicted'],
                   y=df['Actual'],
                   mode ='markers')

trace2 = go.Scatter(
                  x=[df['Predicted'].min(),df['Predicted'].max()] ,
                  y=df['trendline'].sort_values(), 
                  mode='lines',
                  marker=go.Marker(color='rgb(31, 119, 180)'),
                  name='Fit'
                  )

plt.iplot([trace1,trace2])

NameError: name 'go' is not defined

The Mean Squared Error (MSE) is a measure of how close a fitted line is to data points. For every data point, you take the distance vertically from the point to the corresponding y value on the curve fit (the error), and square the value. Then you add up all those values for all data points, and, in the case of a linear fit, divide by the number of points minus two.** The squaring is done so negative values do not cancel positive values. The smaller the Mean Squared Error, the closer the fit is to the data. The MSE has the units squared of whatever is plotted on the vertical axis.

In [24]:
prediction = lr.predict(X_test)

##calculate the mean squared error of the model
mseFull = np.mean((y_test - lr.predict(X_test))**2)
print(mseFull)

13.7333069614


In [25]:
##calculate the score of our model predicted/actual
lr.score(X_test,y_test)

0.78970128695033248

# Feature Selection  

In machine learning and statistics, feature selection, also known as variable selection, attribute selection or variable subset selection, is the process of selecting a subset of relevant features (variables, predictors) for use in model construction.

The central premise when using a feature selection technique is that the data contains many features that are either redundant or irrelevant, and can thus be removed without incurring much loss of information. Redundant or irrelevant features are two distinct notions, since one relevant feature may be redundant in the presence of another relevant feature with which it is strongly correlated.

In [29]:
from sklearn.feature_selection import SelectFromModel


sfm = SelectFromModel(lr,threshold=0.25)
sfm.fit(X_test,y_test)
print(sfm.fit(X_test,y_test))
X_transform = sfm.transform(X_test)

SelectFromModel(estimator=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False),
        prefit=False, threshold=0.25)


In [30]:
lr.fit(X_transform,y_test)
score = lr.score(X_transform,y_test)

In [34]:
X_test

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
173,0.000961,0.000,0.131598,0.0,0.257202,0.547040,0.836251,0.137921,0.173913,0.208015,0.425532,0.996470,0.201711
274,0.000563,0.400,0.218109,1.0,0.127572,0.612569,0.308960,0.268076,0.130435,0.127863,0.531915,1.000000,0.049669
491,0.001117,0.000,1.000000,0.0,0.460905,0.464074,0.987642,0.067155,0.130435,1.000000,0.797872,0.982879,0.450883
72,0.000959,0.000,0.379399,0.0,0.057613,0.479785,0.050463,0.378079,0.130435,0.225191,0.702128,0.984896,0.104581
452,0.057141,0.000,0.646628,0.0,0.674897,0.524238,0.915551,0.112632,1.000000,0.914122,0.808511,0.970220,0.428808
76,0.001070,0.000,0.453446,0.0,0.106996,0.520789,0.737384,0.265766,0.173913,0.402672,0.648936,0.941399,0.282561
316,0.003506,0.000,0.346041,0.0,0.327160,0.450853,0.826982,0.260892,0.130435,0.223282,0.617021,0.984366,0.458057
140,0.003199,0.000,0.785557,0.0,0.491770,0.500671,0.934089,0.043858,0.130435,0.477099,0.914894,0.977760,0.618929
471,0.045320,0.000,0.646628,0.0,0.302469,0.511209,0.904222,0.179114,1.000000,0.914122,0.808511,0.996041,0.307395
500,0.002451,0.000,0.338343,0.0,0.411523,0.472504,0.790937,0.124453,0.217391,0.389313,0.702128,1.000000,0.347682


In [33]:
n_features = sfm.transform(X_test).shape[1]
features = pd.DataFrame({'n_features':n_features,'score':score,'MSE':mseFull},index=[0])
while n_features > 1:
    sfm.threshold += 0.1
    X_transform = sfm.transform(X_test)
    n_features = X_transform.shape[1]
    lr.fit(X_transform,y_test)
    score = lr.score(X_transform,y_test)
    mseFull = np.mean((y_test - lr.predict(X_transform))**2)
    temp = pd.DataFrame({'n_features':n_features,'score':score,'MSE':mseFull},index=[0])
    features = features.append(temp)
    

group_features = features.groupby(features['n_features'],as_index=False).last()
group_features


,n_features,MSE,score
0,1,25.636478,0.607428


From this we can infer that our model performs better the more columns that are included in the linear regression

# Decision Tree Prediction


In [367]:
#As we are predicting a continuous variable, we import the regressor
from sklearn.tree import DecisionTreeRegressor

dtr = DecisionTreeRegressor(random_state = 100,
                               max_depth=3, min_samples_leaf=5)
dtr.fit(X_train, y_train)


predicted = dtr.predict(X_test)

dtr_predictions = pd.DataFrame({'Actual':y_test,'Predicted':predicted})

dtr_predictions.head()



,Actual,Predicted
0,23.6,22.871831
1,32.4,22.871831
2,13.6,12.894565
3,22.8,22.871831
4,16.1,12.894565


In [369]:
#Model score 
score = dtr.score(X_test,y_test)
print(score)


0.778910370136


From this we can see that our basic Decision Tree outperforms the linear regression

## Using cross validation and grid search to improve models

From the decision tree above we can see that there are many differnt options available to set when creating the model. Going through and adjusting each of these by hand to improve our score would take time. This is where cross validation comes in

In [407]:
from sklearn.model_selection import GridSearchCV

params ={'max_depth':[1,2,3,4,5,6,7,8],
         'min_samples_leaf':[1,2,3,4,5]
         }
grid_search = GridSearchCV(dtr,param_grid=params,cv=5)
grid_search.fit(X_train, y_train)


print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

predicted = grid_search.predict(X_test)

grid_predictions = pd.DataFrame({'Actual':y_test,'Predicted':predicted})
score = grid_search.score(X_test,y_test)
print("predicted score against actual: %f" % score)
grid_predictions.head()


Best score: 0.8167652691302683
Best parameters: {'max_depth': 6, 'min_samples_leaf': 3}
predicted score against actual: 0.898348


,Actual,Predicted
0,23.6,20.844961
1,32.4,31.291667
2,13.6,16.823810
3,22.8,23.542857
4,16.1,16.823810


From the above we can see that by using grid_search to help tune the model for us it has had a large impact, improving the model's accuracy by over ten percent

# Random Forest Prediction

In this section we will look at whether a Random Forest model prediction will be more accurate than the linear regression model above. 

In [402]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators= 15, random_state = 100,
                               max_depth=3, min_samples_leaf=5)


rf.fit(X_train, y_train)
predicted = rf.predict(X_test)
rf_predictions = pd.DataFrame({'Actual':y_test,'Predicted':predicted})
rf_score = rf.score(X_test,y_test)
print("predicted score against actual: %f" % rf_score)
rf_predictions.head()

predicted score against actual: 0.824989


,Actual,Predicted
0,23.6,23.539519
1,32.4,25.743848
2,13.6,16.055184
3,22.8,23.874663
4,16.1,15.265623


Grid search on random forest to tune model

In [406]:
params ={'n_estimators':[130,140,150],'max_depth':[20,21,22,23,24,25],
         'min_samples_leaf':[1]
         }
grid_search = GridSearchCV(rf,param_grid=params)
grid_search.fit(X_train, y_train)


print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

predicted = grid_search.predict(X_test)

grid_predictions = pd.DataFrame({'Actual':y_test,'Predicted':predicted})
score = grid_search.score(X_test,y_test)
print("predicted score against actual: %f" % score)
grid_predictions.head()


Best score: 0.8579498759563667
Best parameters: {'max_depth': 23, 'min_samples_leaf': 1, 'n_estimators': 140}
predicted score against actual: 0.886815


,Actual,Predicted
0,23.6,22.313571
1,32.4,30.355000
2,13.6,15.726429
3,22.8,23.316429
4,16.1,16.862857


From the above that although the random forest initially performed better than the decision tree when tuning with grid search the decision tree has created a better model

# Gradient Boosted Regression

In [408]:
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor(n_estimators= 15, random_state = 100,
                               max_depth=3, min_samples_leaf=5)


gbr.fit(X_train, y_train)
predicted = gbr.predict(X_test)
gbr_predictions = pd.DataFrame({'Actual':y_test,'Predicted':predicted})
gbr_score = gbr.score(X_test,y_test)
print("predicted score against actual: %f" % gbr_score)
gbr_predictions.head()


predicted score against actual: 0.828524


,Actual,Predicted
0,23.6,22.897290
1,32.4,27.030809
2,13.6,17.457687
3,22.8,23.381114
4,16.1,17.428375


In [411]:
params ={'n_estimators':[50,100,130],'max_depth':[20,21,22,23],
         'min_samples_leaf':[1,2,3,4,5],'learning_rate':[0.1,0.05,0.02]
         }
grid_search = GridSearchCV(gbr,param_grid=params)
grid_search.fit(X_train, y_train)


print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

predicted = grid_search.predict(X_test)

grid_predictions = pd.DataFrame({'Actual':y_test,'Predicted':predicted})
score = grid_search.score(X_test,y_test)
print("predicted score against actual: %f" % score)
grid_predictions.head()

Best score: 0.8489300863222985
Best parameters: {'learning_rate': 0.1, 'max_depth': 21, 'min_samples_leaf': 3, 'n_estimators': 130}
predicted score against actual: 0.916756


,Actual,Predicted
0,23.6,23.295662
1,32.4,31.242559
2,13.6,18.104824
3,22.8,23.785077
4,16.1,17.252985


# Neural network regression model

In [416]:
from sklearn.neural_network import MLPRegressor

mlp = MLPRegressor(hidden_layer_sizes=(30,30,30))

mlp.fit(X_train,y_train)

predicted = mlp.predict(X_test)
mlp_predictions = pd.DataFrame({'Actual':y_test,'Predicted':predicted})
mlp_score = mlp.score(X_test,y_test)
print("predicted score against actual: %f" % mlp_score)
mlp_predictions.head()


predicted score against actual: 0.847226


,Actual,Predicted
0,23.6,28.548549
1,32.4,35.951723
2,13.6,14.443148
3,22.8,23.728831
4,16.1,18.489436


In [424]:
from sklearn.model_selection import GridSearchCV

params ={'learning_rate':['constant', 'adaptive']
         }
grid_search = GridSearchCV(mlp,param_grid=params,cv=5)
grid_search.fit(X_train, y_train)


print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

predicted = grid_search.predict(X_test)

grid_predictions = pd.DataFrame({'Actual':y_test,'Predicted':predicted})
score = grid_search.score(X_test,y_test)
print("predicted score against actual: %f" % score)
grid_predictions.head()

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:563: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:563: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:563: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:563: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:563: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations re

Best score: 0.7193964448816771
Best parameters: {'learning_rate': 'constant'}
predicted score against actual: 0.848918


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:563: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.



,Actual,Predicted
0,23.6,26.940479
1,32.4,34.782689
2,13.6,16.350471
3,22.8,24.867512
4,16.1,18.945252
